<a href="https://colab.research.google.com/github/patrickbryant1/MoLPC/blob/master/MoLPC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>